## Vorbereitung
1. Import der Bibliotheken
2. Import der Dateien + Merge
3. Sortierung des df

In [33]:
# Imports

%pip install seaborn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.


In [34]:
# Umsatzdaten laden
df_umsatz = pd.read_csv('data/umsatzdaten_gekuerzt.csv')
df_test = pd.read_csv('test.csv')
df_holidays = pd.read_csv('holidays_germany.csv')
df_wetter = pd.read_csv('data/wetter.csv')
df_kiwo = pd.read_csv('data/kiwo.csv')
df_wetter_dwd = pd.read_csv('additional-data/Klimadaten_DWD_Kiel_Holtenau.csv')

# Datum konvertieren
df_umsatz['Datum'] = pd.to_datetime(df_umsatz['Datum'], errors='coerce')
df_test['Datum'] = pd.to_datetime(df_test['Datum'], errors='coerce')
df_wetter_dwd['Datum'] = pd.to_datetime(df_test['Datum'], errors='coerce')

# Zusammenführen 
df_umsatz = pd.concat([df_umsatz, df_test], ignore_index=True)

# Nach Datum sortieren
df_umsatz = df_umsatz.sort_values(by='Datum')

df_wetter['Datum'] = pd.to_datetime(df_wetter['Datum'], errors='coerce')
df_kiwo['Datum'] = pd.to_datetime(df_kiwo['Datum'], errors='coerce')

df_holidays['date'] = pd.to_datetime(df_holidays['date'], errors='coerce')
df_holidays = df_holidays.rename(columns={'date': 'Datum'})

merged_wetter_umsatz = pd.merge(df_umsatz, df_wetter, on='Datum', how='left')
merged_kiwo = pd.merge(merged_wetter_umsatz, df_kiwo, on='Datum', how='left')
df_merged = pd.merge(merged_kiwo, df_holidays, on='Datum', how='left')

df_merged = df_merged.rename(columns={'holiday': 'Feiertage'})

print("Min:", df_merged['Datum'].min())
print("Max:", df_merged['Datum'].max())

print("Min:", df_merged['Wettercode'].min())
print("Max:", df_merged['Wettercode'].max())

display(df_merged.head())
print(df_merged.shape)
print(df_merged['Datum'].is_monotonic_increasing)


Min: 2013-07-01 00:00:00
Max: 2019-07-30 00:00:00
Min: 0.0
Max: 95.0


,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Feiertage
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,NaN,NaN
1,1307013,2013-07-01,3,201.198426,6.0,17.8375,15.0,20.0,NaN,NaN
2,1307014,2013-07-01,4,65.890169,6.0,17.8375,15.0,20.0,NaN,NaN
3,1307015,2013-07-01,5,317.475875,6.0,17.8375,15.0,20.0,NaN,NaN
4,1307012,2013-07-01,2,535.856285,6.0,17.8375,15.0,20.0,NaN,NaN


(11164, 10)
True


## Data Cleaning - Option: Fehlende Daten löschen

In [35]:
# Data Cleaning vorher

# Wetter- und Feature-Daten bereinigen
#df_merged['Temperatur'] = df_merged['Temperatur'].fillna(df_merged['Temperatur'].mean)
# df_merged['Temperatur'] = df_merged['Temperatur'].fillna(df_merged['Temperatur'].mean())
# df_merged['KielerWoche'] = df_merged['KielerWoche'].fillna(0)
# df_merged['Wettercode'] = df_merged['Wettercode'].fillna(0)
# df_merged['Bewoelkung'] = df_merged['Bewoelkung'].fillna(df_merged['Bewoelkung'].mean())
#df_merged['Windgeschwindigkeit'] = df_merged['Windgeschwindigkeit'].fillna(df_merged['Windgeschwindigkeit'].mean())
#df_merged['Feiertage'] = df_merged['Feiertage'].fillna('Kein Feiertag')

In [36]:
df_merged['Datum'] = pd.to_datetime(df_merged['Datum'], errors='coerce')
df_merged['Monat'] = df_merged['Datum'].dt.month
df_merged['Wochentag'] = df_merged['Datum'].dt.weekday
df_merged = df_merged.drop_duplicates()

df_with_umsatz = df_merged[df_merged['Umsatz'].notna()].copy()
df_without_umsatz = df_merged[df_merged['Umsatz'].isna()].copy()

print(f"Zeilen mit Umsatz: {len(df_with_umsatz)}")
print(f"Zeilen ohne Umsatz (zum Vorhersagen): {len(df_without_umsatz)}")

print("Min:", df_merged['Datum'].min())
print("Max:", df_merged['Datum'].max())

print("Fehlende Werte pro Spalte:")
print(df_merged.isna().sum())
df_merged.head()

Zeilen mit Umsatz: 9334
Zeilen ohne Umsatz (zum Vorhersagen): 1830
Min: 2013-07-01 00:00:00
Max: 2019-07-30 00:00:00
Fehlende Werte pro Spalte:
id                         0
Datum                      0
Warengruppe                0
Umsatz                  1830
Bewoelkung               135
Temperatur                81
Windgeschwindigkeit       81
Wettercode              2662
KielerWoche            10896
Feiertage              11059
Monat                      0
Wochentag                  0
dtype: int64


,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Feiertage,Monat,Wochentag
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,NaN,NaN,7,0
1,1307013,2013-07-01,3,201.198426,6.0,17.8375,15.0,20.0,NaN,NaN,7,0
2,1307014,2013-07-01,4,65.890169,6.0,17.8375,15.0,20.0,NaN,NaN,7,0
3,1307015,2013-07-01,5,317.475875,6.0,17.8375,15.0,20.0,NaN,NaN,7,0
4,1307012,2013-07-01,2,535.856285,6.0,17.8375,15.0,20.0,NaN,NaN,7,0


### Analyse der fehlenden Temperatur-Werte

In [37]:
df_merged['Temperatur'].isna().sum(), df_merged['Temperatur'].dtype
df_merged[['Datum','Temperatur']].head(5)
df_merged[['Datum','Temperatur']].tail(5)

missing = df_merged[df_merged['Temperatur'].isna()].copy()
missing[['Datum','Temperatur']].head(20)  # erhalte ein Gefühl

# Zeige für die fehlenden Zeilen die Vor- und Nachwerte:
display(pd.concat([
    missing[['Datum','Temperatur']],
    df_merged['Temperatur'].shift(1).loc[missing.index].rename('temp_minus1'),
    df_merged['Temperatur'].shift(-1).loc[missing.index].rename('temp_plus1')
], axis=1).head(30))

isna = df_merged['Temperatur'].isna().astype(int)
# Laufende Gruppenlängen der NaNs:
g = (isna != isna.shift()).cumsum()
na_groups = isna.groupby(g).sum()
# Filter nur die Gruppen >0 (also nur die Na-Gruppen); zeige Verteilung:
na_group_lengths = na_groups[na_groups>0].value_counts().sort_index()
# In eine DataFrame umwandeln und Spaltenüberschriften setzen 
df_na_group_lengths = na_group_lengths.reset_index()
df_na_group_lengths.columns = ['Länge der NaN-Gruppe Temperatur', 'Häufigkeiten']  
display(df_na_group_lengths)

,Datum,Temperatur,temp_minus1,temp_plus1
6348,2016-12-11,NaN,9.5000,NaN
6349,2016-12-11,NaN,NaN,NaN
6350,2016-12-11,NaN,NaN,NaN
6351,2016-12-11,NaN,NaN,NaN
6352,2016-12-11,NaN,NaN,NaN
6353,2016-12-11,NaN,NaN,4.8750
7813,2017-10-04,NaN,14.2625,NaN
7814,2017-10-04,NaN,NaN,NaN
7815,2017-10-04,NaN,NaN,NaN
7816,2017-10-04,NaN,NaN,NaN


,Länge der NaN-Gruppe Temperatur,Häufigkeiten
0,5,1
1,6,1
2,10,4
3,30,1


--> es ergibt keinen Sinn die Mittelwerte der angrenzenden Tage zu nehmen, da die Werte immer als Reihe fehlen

### Analyse Windgeschwindigkeit

In [38]:
df_merged['Windgeschwindigkeit'].isna().sum(), df_merged['Windgeschwindigkeit'].dtype
df_merged[['Datum','Windgeschwindigkeit']].head(5)
df_merged[['Datum','Windgeschwindigkeit']].tail(5)

missing = df_merged[df_merged['Windgeschwindigkeit'].isna()].copy()
missing[['Datum','Windgeschwindigkeit']].head(20)  # erhalte ein Gefühl

# Zeige für die fehlenden Zeilen die Vor- und Nachwerte:
display(pd.concat([
    missing[['Datum','Windgeschwindigkeit']],
    df_merged['Windgeschwindigkeit'].shift(1).loc[missing.index].rename('temp_minus1'),
    df_merged['Windgeschwindigkeit'].shift(-1).loc[missing.index].rename('temp_plus1')
], axis=1).head(30))

isna = df_merged['Windgeschwindigkeit'].isna().astype(int)
# Laufende Gruppenlängen der NaNs:
g = (isna != isna.shift()).cumsum()
na_groups = isna.groupby(g).sum()
# Filter nur die Gruppen >0 (also nur die Na-Gruppen); zeige Verteilung:
na_group_lengths = na_groups[na_groups>0].value_counts().sort_index()
# In eine DataFrame umwandeln und Spaltenüberschriften setzen 
df_na_group_lengths = na_group_lengths.reset_index()
df_na_group_lengths.columns = ['Länge der NaN-Gruppe Windgeschwindigkeit', 'Häufigkeiten']  
display(df_na_group_lengths)

,Datum,Windgeschwindigkeit,temp_minus1,temp_plus1
6348,2016-12-11,NaN,10.0,NaN
6349,2016-12-11,NaN,NaN,NaN
6350,2016-12-11,NaN,NaN,NaN
6351,2016-12-11,NaN,NaN,NaN
6352,2016-12-11,NaN,NaN,NaN
6353,2016-12-11,NaN,NaN,4.0
7813,2017-10-04,NaN,14.0,NaN
7814,2017-10-04,NaN,NaN,NaN
7815,2017-10-04,NaN,NaN,NaN
7816,2017-10-04,NaN,NaN,NaN


,Länge der NaN-Gruppe Windgeschwindigkeit,Häufigkeiten
0,5,1
1,6,1
2,10,4
3,30,1


In [39]:
# Vergleich: Fehlende Werte in Temperatur und Windgeschwindigkeit
# 1. Überlappung prüfen: Welche Zeilen haben BEIDE Spalten als NaN?
both_missing = df_merged[(df_merged['Temperatur'].isna()) & (df_merged['Windgeschwindigkeit'].isna())]
print(f"Zeilen, wo BEIDE (Temperatur & Windgeschwindigkeit) fehlen: {len(both_missing)}")

# 2. Nur Temperatur fehlt (aber Windgeschwindigkeit vorhanden)
only_temp_missing = df_merged[(df_merged['Temperatur'].isna()) & (df_merged['Windgeschwindigkeit'].notna())]
print(f"Zeilen, wo NUR Temperatur fehlt: {len(only_temp_missing)}")

# 3. Nur Windgeschwindigkeit fehlt (aber Temperatur vorhanden)
only_wind_missing = df_merged[(df_merged['Windgeschwindigkeit'].isna()) & (df_merged['Temperatur'].notna())]
print(f"Zeilen, wo NUR Windgeschwindigkeit fehlt: {len(only_wind_missing)}")

# 4. Zusammenfassung als Tabelle
summary = pd.DataFrame({
    'Kategorie': ['Beide fehlen', 'Nur Temperatur fehlt', 'Nur Windgeschwindigkeit fehlt'],
    'Anzahl': [len(both_missing), len(only_temp_missing), len(only_wind_missing)]
})
display(summary)

# 5. Beispiel: Zeige ein paar Zeilen, wo beide fehlen
print("\nBeispiele (erste 10 Zeilen, wo beide fehlen):")
display(both_missing[['Datum', 'Temperatur', 'Windgeschwindigkeit']].head(10))

Zeilen, wo BEIDE (Temperatur & Windgeschwindigkeit) fehlen: 81
Zeilen, wo NUR Temperatur fehlt: 0
Zeilen, wo NUR Windgeschwindigkeit fehlt: 0


,Kategorie,Anzahl
0,Beide fehlen,81
1,Nur Temperatur fehlt,0
2,Nur Windgeschwindigkeit fehlt,0



Beispiele (erste 10 Zeilen, wo beide fehlen):


,Datum,Temperatur,Windgeschwindigkeit
6348,2016-12-11,NaN,NaN
6349,2016-12-11,NaN,NaN
6350,2016-12-11,NaN,NaN
6351,2016-12-11,NaN,NaN
6352,2016-12-11,NaN,NaN
6353,2016-12-11,NaN,NaN
7813,2017-10-04,NaN,NaN
7814,2017-10-04,NaN,NaN
7815,2017-10-04,NaN,NaN
7816,2017-10-04,NaN,NaN


Löschen der 81 Zeilen mit fehlenden Temperatur- und Windgeschwindigkeitsdaten; 81 Zeilen = 15 Tage

### Option 1: Löschen

In [40]:
# Zeilen mit fehlenden Werten löschen - verschiedene Optionen

print(f"Originale Anzahl Zeilen: {len(df_merged)}")
print(f"Originale fehlende Werte:\n{df_merged.isna().sum()}\n")

# Option 2: Nur Zeilen mit NaN in SPEZIFISCHEN Spalten löschen
# Ausnahme: Behalte Zeilen im Zeitraum 01.08.2018 - 30.07.2019, auch wenn dort NaNs auftreten
mask_keep = (df_merged['Datum'] >= pd.to_datetime('2018-08-01')) & (df_merged['Datum'] <= pd.to_datetime('2019-07-30'))
mask_missing = df_merged[['Temperatur', 'Windgeschwindigkeit']].isna().any(axis=1)
mask_drop = mask_missing & (~mask_keep)
print(f"Originale Anzahl Zeilen: {len(df_merged)}")
print(f"Zu löschende Zeilen: {mask_drop.sum()}")
df_merged = df_merged.loc[~mask_drop].copy()
print(f"Nach bedingtem dropna: {len(df_merged)} Zeilen")

Originale Anzahl Zeilen: 11164
Originale fehlende Werte:
id                         0
Datum                      0
Warengruppe                0
Umsatz                  1830
Bewoelkung               135
Temperatur                81
Windgeschwindigkeit       81
Wettercode              2662
KielerWoche            10896
Feiertage              11059
Monat                      0
Wochentag                  0
dtype: int64

Originale Anzahl Zeilen: 11164
Zu löschende Zeilen: 16
Nach bedingtem dropna: 11148 Zeilen


In [41]:
# # Option 2: Nur Zeilen mit NaN in SPEZIFISCHEN Spalten löschen
# df_merged = df_merged.dropna(subset=['Temperatur', 'Windgeschwindigkeit'])
# print(f"Nach dropna: {len(df_merged)} Zeilen")

In [42]:
# Feature-Daten bereinigen
df_merged['KielerWoche'] = df_merged['KielerWoche'].fillna(0)
df_merged['Feiertage'] = df_merged['Feiertage'].fillna(0)

df_with_umsatz = df_merged[df_merged['Umsatz'].notna()].copy()
df_without_umsatz = df_merged[df_merged['Umsatz'].isna()].copy()

print(f"Zeilen mit Umsatz: {len(df_with_umsatz)}")
print(f"Zeilen ohne Umsatz (zum Vorhersagen): {len(df_without_umsatz)}")

df_merged.to_csv("data/df_merged_new.csv", index=False)

print("Min:", df_merged['Datum'].min())
print("Max:", df_merged['Datum'].max())


print(df_merged.isna().sum())
df_merged.head()

Zeilen mit Umsatz: 9318
Zeilen ohne Umsatz (zum Vorhersagen): 1830


Min: 2013-07-01 00:00:00
Max: 2019-07-30 00:00:00
id                        0
Datum                     0
Warengruppe               0
Umsatz                 1830
Bewoelkung              119
Temperatur               65
Windgeschwindigkeit      65
Wettercode             2646
KielerWoche               0
Feiertage                 0
Monat                     0
Wochentag                 0
dtype: int64


,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Feiertage,Monat,Wochentag
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,0.0,0,7,0
1,1307013,2013-07-01,3,201.198426,6.0,17.8375,15.0,20.0,0.0,0,7,0
2,1307014,2013-07-01,4,65.890169,6.0,17.8375,15.0,20.0,0.0,0,7,0
3,1307015,2013-07-01,5,317.475875,6.0,17.8375,15.0,20.0,0.0,0,7,0
4,1307012,2013-07-01,2,535.856285,6.0,17.8375,15.0,20.0,0.0,0,7,0


## Fehlende Temperaturdaten ergänzen

In [43]:
print(df_merged.head())

        id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01            1  148.828353         6.0     17.8375   
1  1307013 2013-07-01            3  201.198426         6.0     17.8375   
2  1307014 2013-07-01            4   65.890169         6.0     17.8375   
3  1307015 2013-07-01            5  317.475875         6.0     17.8375   
4  1307012 2013-07-01            2  535.856285         6.0     17.8375   

   Windgeschwindigkeit  Wettercode  KielerWoche Feiertage  Monat  Wochentag  
0                 15.0        20.0          0.0         0      7          0  
1                 15.0        20.0          0.0         0      7          0  
2                 15.0        20.0          0.0         0      7          0  
3                 15.0        20.0          0.0         0      7          0  
4                 15.0        20.0          0.0         0      7          0  


In [44]:
# Ergänze fehlende Temperaturdaten aus DWD-Daten
df_wetter_dwd = pd.read_csv('data/df_wetterdaten_dwd.csv')
df_wetter_dwd['Datum'] = pd.to_datetime(df_wetter_dwd['Datum'], errors='coerce')

# Ergänze fehlende Angaben der Temperatur aus DWD-Daten

# Merge über gemeinsame Spalte (z.B. 'Datum')
df_merged = df_merged.merge(df_wetter_dwd[['Datum', 'Temperatur']], 
                       on='Datum', 
                       how='left', 
                       suffixes=('', 'DWD'))

# Fehlende Werte ergänzen
df_merged['Temperatur'] = df_merged['Temperatur'].fillna(df_merged['TemperaturDWD'])

print ('Fehlende Temperaturwerte:') 
print(df_merged['Temperatur'].isna().sum())  # Überprüfe, ob noch fehlende Werte vorhanden sind

print(df_merged[df_merged['Temperatur'].isna()][['Datum', 'Temperatur', 'TemperaturDWD']])

Fehlende Temperaturwerte:
0
Empty DataFrame
Columns: [Datum, Temperatur, TemperaturDWD]
Index: []


In [45]:
print(df_merged.isna().sum())

id                        0
Datum                     0
Warengruppe               0
Umsatz                 1830
Bewoelkung              119
Temperatur                0
Windgeschwindigkeit      65
Wettercode             2646
KielerWoche               0
Feiertage                 0
Monat                     0
Wochentag                 0
TemperaturDWD             0
dtype: int64


In [46]:
# Lösche Spalte mit DWD-Temperaturdaten nach dem Ergänzen
df_merged = df_merged.drop(columns=['TemperaturDWD'])
print(df_merged.head())

        id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01            1  148.828353         6.0     17.8375   
1  1307013 2013-07-01            3  201.198426         6.0     17.8375   
2  1307014 2013-07-01            4   65.890169         6.0     17.8375   
3  1307015 2013-07-01            5  317.475875         6.0     17.8375   
4  1307012 2013-07-01            2  535.856285         6.0     17.8375   

   Windgeschwindigkeit  Wettercode  KielerWoche Feiertage  Monat  Wochentag  
0                 15.0        20.0          0.0         0      7          0  
1                 15.0        20.0          0.0         0      7          0  
2                 15.0        20.0          0.0         0      7          0  
3                 15.0        20.0          0.0         0      7          0  
4                 15.0        20.0          0.0         0      7          0  


## Ergänze Sonnen- und Regenstunden

In [47]:
# # Merge df_wetterdaten_dwd Sonnenscheindauer aus DWD mit dem Haupt-Datensatz
# df_merged = pd.merge(
#     df_merged, 
#     df_wetter_dwd[['Datum', 'NiederschlagTagessummeInMM']], 
#     on='Datum', 
#     how='left'
# )

# print(df_merged.head())

# # -999.0 durch NaN ersetzen
# df_merged['Sonnenscheindauer'] = df_merged['Sonnenscheindauer'].replace(-999.0, np.nan)
# print('Fehlende Sonnenscheindauer Werte:')
# print(df_merged['Sonnenscheindauer'].isna().sum())

## One-Hot-Encoding: Categorical Data

1. Wettercode
2. Warengruppe + Wochentag + Monat + Bewoelkung + Feiertage 

### 1. Wettercode

In [48]:
print(df_merged['Wettercode'].value_counts())

Wettercode
61.0    2777
21.0    1193
0.0     1044
10.0     865
5.0      774
63.0     580
20.0     307
95.0     172
22.0     115
65.0     111
45.0     106
71.0      91
28.0      74
49.0      53
53.0      48
73.0      48
77.0      45
69.0      27
75.0      25
68.0      22
17.0      10
79.0       5
3.0        5
55.0       5
Name: count, dtype: int64


In [49]:
# Debug-Ausgabe
print("dtype vor Änderung:", df_merged['Wettercode'].dtype)

# Falls noch kein category dtype: konvertieren
if not pd.api.types.is_categorical_dtype(df_merged['Wettercode']):
    df_merged['Wettercode'] = df_merged['Wettercode'].astype('category')

# Jetzt sicher Kategorien hinzufügen und NaNs ersetzen
df_merged['Wettercode'] = df_merged['Wettercode'].cat.add_categories(['Keine Daten'])
df_merged['Wettercode'] = df_merged['Wettercode'].fillna('Keine Daten')

print("dtype nach Änderung:", df_merged['Wettercode'].dtype)
print(df_merged['Wettercode'].value_counts())

dtype vor Änderung: float64
dtype nach Änderung: category
Wettercode
61.0           2777
Keine Daten    2646
21.0           1193
0.0            1044
10.0            865
5.0             774
63.0            580
20.0            307
95.0            172
22.0            115
65.0            111
45.0            106
71.0             91
28.0             74
49.0             53
53.0             48
73.0             48
77.0             45
69.0             27
75.0             25
68.0             22
17.0             10
3.0               5
55.0              5
79.0              5
Name: count, dtype: int64


/tmp/ipykernel_31496/3099108898.py:5: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df_merged['Wettercode']):


In [50]:
print("Spalten in df_merged:", list(df_merged.columns))
if 'Wettercode' not in df_merged.columns:
    print("FEHLER: Spalte 'Wettercode' fehlt. Zeige ersten 5 Zeilen:")
    display(df_merged.head())

Spalten in df_merged: ['id', 'Datum', 'Warengruppe', 'Umsatz', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode', 'KielerWoche', 'Feiertage', 'Monat', 'Wochentag']


In [51]:
# Wettercodes in Kategorien gruppieren mit Mapping

wettercode_mapping = {
    0.0: 'Klar',
    5.0: 'Dunst',
    10.0: 'Dunst',
    17.0: 'Extremwetter',
    20.0: 'NachSchlechtemWetter',
    21.0: 'NachSchlechtemWetter',
    22.0: 'NachSchlechtemWetter',
    28.0: 'NachSchlechtemWetter',
    45.0: 'Nebel',
    49.0: 'Nebel',
    53.0: 'Regen',
    55.0: 'Regen',
    61.0: 'Regen',
    63.0: 'Regen',
    65.0: 'Regen',
    68.0: 'Regen',
    69.0: 'Regen',
    71.0: 'Schnee',          
    73.0: 'Schnee',          
    75.0: 'Schnee',
    77.0: 'Schnee',
    79.0: 'Extremwetter',
    95.0: 'Extremwetter',
    'KeineDaten': 'KeineDaten'
}

df_merged['Wettercode'] = df_merged['Wettercode'].map(wettercode_mapping)


# Häufigkeit der Kategorien zählen
value_counts = df_merged['Wettercode'].value_counts()
print("Ursprüngliche Verteilung:")
print(value_counts)

# # Schwellenwert festlegen - hier 36
# min_haeufigkeit = 36

# # Kategorien identifizieren, die seltener als 36 mal vorkommen
# seltene_kategorien = value_counts[value_counts < min_haeufigkeit].index.tolist()

# print(f"\nSeltene Kategorien (< {min_haeufigkeit} Vorkommen):")
# print(seltene_kategorien)

# # Seltene Kategorien durch 'Sonstige' ersetzen
# df_merged['Wettercode'] = df_merged['Wettercode'].apply(
#     lambda x: 'Sonstige' if x in seltene_kategorien else x
# )

# print("\nNeue Verteilung nach Zusammenfassung:")
# print(df_merged['Wettercode'].value_counts())


#alle_kategorien = ['Klar','Dunst','Nebel','Sprühregen_oder_Schneegriesel_aufgehoert',
#                    'Regen_aufgehoert','Schnee_aufgehoert','Nebel_aufgehoert','Nebel+Reifansatz',
#                    'andauernder_Sprühregen','Mässiger_Regen','Schneeregen','Schneefall',
#                    'Schneegriesel','Eiskörner','Gewitter','Keine_Daten']  

# df_merged['Wettercode'] = pd.Categorical(df_merged['Wettercode'], categories=wettercode_mapping.values())

# One-Hot-Encoding auf die neuen Kategorien
df_merged = pd.get_dummies(df_merged, columns=['Wettercode'], dtype=int, prefix='Wettercode', prefix_sep='')

print("Wetterkategorien-Verteilung:")
#print(df_merged['Wettercode'].value_counts())

print(df_merged.head())

Ursprüngliche Verteilung:
Wettercode
Regen                   3570
NachSchlechtemWetter    1689
Dunst                   1639
Klar                    1044
Schnee                   209
Extremwetter             187
Nebel                    159
Name: count, dtype: int64
Wetterkategorien-Verteilung:
        id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01            1  148.828353         6.0     17.8375   
1  1307013 2013-07-01            3  201.198426         6.0     17.8375   
2  1307014 2013-07-01            4   65.890169         6.0     17.8375   
3  1307015 2013-07-01            5  317.475875         6.0     17.8375   
4  1307012 2013-07-01            2  535.856285         6.0     17.8375   

   Windgeschwindigkeit  KielerWoche Feiertage  Monat  Wochentag  \
0                 15.0          0.0         0      7          0   
1                 15.0          0.0         0      7          0   
2                 15.0          0.0         0      7       

#### Test, ob kein Wert in Wetterspalte > 1

In [52]:
# Nach pd.get_dummies() — prüfe One-Hot-Encoding Integrität
wetter_spalten = [col for col in df_merged.columns if col.startswith('Wetter_')]

print(f"Anzahl Wetter-Spalten: {len(wetter_spalten)}")
print(f"Wetter-Spalten: {wetter_spalten}\n")

# Prüfe: max Wert in jeder Wetter-Spalte
for col in wetter_spalten:
    max_val = df_merged[col].max()
    min_val = df_merged[col].min()
    print(f"{col}: min={min_val}, max={max_val}")

# Zusammenfassung: alle Wetter-Spalten prüfen
if (df_merged[wetter_spalten] <= 1).all().all():
    print("\n✓ OK: Alle Wetter-Spalten enthalten nur Werte ≤ 1")
else:
    print("\n✗ FEHLER: Einige Wetter-Spalten enthalten Werte > 1")
    print(df_merged[wetter_spalten].describe())
    
# Zusätzlich: Zeige Zeilen, wo Summe pro Zeile != 1 ist
wetter_sum = df_merged[wetter_spalten].sum(axis=1)
if (wetter_sum == 1).all():
    print("✓ OK: Jede Zeile hat genau eine 1 in den Wetter-Spalten")
else:
    print(f"⚠ WARNUNG: {(wetter_sum != 1).sum()} Zeilen haben Summe ≠ 1")
    print(wetter_sum.value_counts())

Anzahl Wetter-Spalten: 0
Wetter-Spalten: []


✓ OK: Alle Wetter-Spalten enthalten nur Werte ≤ 1
⚠ WARNUNG: 11148 Zeilen haben Summe ≠ 1
0.0    11148
Name: count, dtype: int64


### 2. Weitere

#### Warengruppe

In [53]:
# Debug-Ausgabe
print("dtype vor Änderung:", df_merged['Warengruppe'].dtype)

# Falls noch kein category dtype: konvertieren
if not pd.api.types.is_categorical_dtype(df_merged['Warengruppe']):
    df_merged['Warengruppe'] = df_merged['Warengruppe'].astype('category')


print("dtype nach Änderung:", df_merged['Warengruppe'].dtype)
print(df_merged['Warengruppe'].value_counts())

dtype vor Änderung: int64
dtype nach Änderung: category
Warengruppe
1    2171
2    2171
3    2171
5    2171
4    2117
6     347
Name: count, dtype: int64


/tmp/ipykernel_31496/2051492849.py:5: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df_merged['Warengruppe']):


In [54]:
# Warengruppen in Kategorien gruppieren mit Mapping

Warengruppe_mapping = {
    1: 'Bread',
    2: 'Rolls',
    3: 'Croissants',
    4: 'Confectionery',
    5: 'Cake',
    6: 'SeasonalBread',          
}

df_merged['Warengruppe'] = df_merged['Warengruppe'].map(Warengruppe_mapping)


# One-Hot-Encoding auf die neuen Kategorien
df_merged = pd.get_dummies(df_merged, columns=['Warengruppe'], dtype=int, prefix='Warengruppe', prefix_sep='')

print("Warengruppe-Verteilung:")
#print(df_merged['Wettercode'].value_counts())

print(df_merged.head())



Warengruppe-Verteilung:
        id      Datum      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01  148.828353         6.0     17.8375   
1  1307013 2013-07-01  201.198426         6.0     17.8375   
2  1307014 2013-07-01   65.890169         6.0     17.8375   
3  1307015 2013-07-01  317.475875         6.0     17.8375   
4  1307012 2013-07-01  535.856285         6.0     17.8375   

   Windgeschwindigkeit  KielerWoche Feiertage  Monat  Wochentag  ...  \
0                 15.0          0.0         0      7          0  ...   
1                 15.0          0.0         0      7          0  ...   
2                 15.0          0.0         0      7          0  ...   
3                 15.0          0.0         0      7          0  ...   
4                 15.0          0.0         0      7          0  ...   

   WettercodeNachSchlechtemWetter  WettercodeNebel  WettercodeRegen  \
0                               1                0                0   
1                               1 

#### Wochentag

In [55]:
# Debug-Ausgabe
print("dtype vor Änderung:", df_merged['Wochentag'].dtype)

# Falls noch kein category dtype: konvertieren
if not pd.api.types.is_categorical_dtype(df_merged['Wochentag']):
    df_merged['Wochentag'] = df_merged['Wochentag'].astype('category')


print("dtype nach Änderung:", df_merged['Wochentag'].dtype)
print(df_merged['Wochentag'].value_counts())

# Wochentage in Kategorien gruppieren mit Mapping

Wochentag_mapping = {
    0: 'UnterDerWoche',
    1: 'UnterDerWoche',
    2: 'UnterDerWoche',
    3: 'UnterDerWoche',
    4: 'Freitag',
    5: 'Samstag',
    6: 'Sonntag',          
}

df_merged['Wochentag'] = df_merged['Wochentag'].map(Wochentag_mapping)


# One-Hot-Encoding auf die neuen Kategorien
df_merged = pd.get_dummies(df_merged, columns=['Wochentag'], dtype=int, prefix='Wochentag', prefix_sep='')

print("Wochentag-Verteilung:")
#print(df_merged['Wettercode'].value_counts())

print(df_merged.head())



dtype vor Änderung: int32
dtype nach Änderung: category
Wochentag
6    1604
5    1604
1    1602
3    1598
0    1591
4    1575
2    1574
Name: count, dtype: int64
Wochentag-Verteilung:
        id      Datum      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01  148.828353         6.0     17.8375   
1  1307013 2013-07-01  201.198426         6.0     17.8375   
2  1307014 2013-07-01   65.890169         6.0     17.8375   
3  1307015 2013-07-01  317.475875         6.0     17.8375   
4  1307012 2013-07-01  535.856285         6.0     17.8375   

   Windgeschwindigkeit  KielerWoche Feiertage  Monat  WettercodeDunst  ...  \
0                 15.0          0.0         0      7                0  ...   
1                 15.0          0.0         0      7                0  ...   
2                 15.0          0.0         0      7                0  ...   
3                 15.0          0.0         0      7                0  ...   
4                 15.0          0.0         0      7       

/tmp/ipykernel_31496/3755633557.py:5: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df_merged['Wochentag']):


#### Monat

In [56]:
# Debug-Ausgabe
print("dtype vor Änderung:", df_merged['Monat'].dtype)

# Falls noch kein category dtype: konvertieren
if not pd.api.types.is_categorical_dtype(df_merged['Monat']):
    df_merged['Monat'] = df_merged['Monat'].astype('category')


print("dtype nach Änderung:", df_merged['Monat'].dtype)
print(df_merged['Monat'].value_counts())

# Monate in Kategorien gruppieren mit Mapping

Wochentag_mapping = {
    1: 'Januar',
    2: 'Februar',
    3: 'Maerz',
    4: 'April',
    5: 'Mai',
    6: 'Juni',
    7: 'Juli',
    8: 'August',
    9: 'September',
    10: 'Oktober',
    11: 'November',
    12: 'Dezember',                  
}

df_merged['Monat'] = df_merged['Monat'].map(Wochentag_mapping)


# One-Hot-Encoding auf die neuen Kategorien
df_merged = pd.get_dummies(df_merged, columns=['Monat'], dtype=int, prefix='', prefix_sep='')

print("Monat-Verteilung:")


print(df_merged.head())



dtype vor Änderung: int32
dtype nach Änderung: category
Monat
11    1066
7     1049
12     996
10     917
8      911
3      909
1      906
9      897
6      895
5      886
4      871
2      845
Name: count, dtype: int64
Monat-Verteilung:
        id      Datum      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01  148.828353         6.0     17.8375   
1  1307013 2013-07-01  201.198426         6.0     17.8375   
2  1307014 2013-07-01   65.890169         6.0     17.8375   
3  1307015 2013-07-01  317.475875         6.0     17.8375   
4  1307012 2013-07-01  535.856285         6.0     17.8375   

   Windgeschwindigkeit  KielerWoche Feiertage  WettercodeDunst  \
0                 15.0          0.0         0                0   
1                 15.0          0.0         0                0   
2                 15.0          0.0         0                0   
3                 15.0          0.0         0                0   
4                 15.0          0.0         0                0   

/tmp/ipykernel_31496/1507491622.py:5: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df_merged['Monat']):


#### Bewölkung

In [57]:
# Debug-Ausgabe
print("dtype vor Änderung:", df_merged['Bewoelkung'].dtype)

# Falls noch kein category dtype: konvertieren
if not pd.api.types.is_categorical_dtype(df_merged['Bewoelkung']):
    df_merged['Bewoelkung'] = df_merged['Bewoelkung'].astype('category')

print("dtype nach Änderung:", df_merged['Bewoelkung'].dtype)
print(df_merged['Bewoelkung'].value_counts())


dtype vor Änderung: float64
dtype nach Änderung: category
Bewoelkung
7.0    3478
6.0    1529
0.0    1428
5.0    1064
8.0     804
4.0     754
3.0     688
1.0     665
2.0     619
Name: count, dtype: int64


/tmp/ipykernel_31496/2772964761.py:5: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df_merged['Bewoelkung']):


In [58]:
# Bewoelkung in Kategorien gruppieren mit Mapping

Bewoelkung_mapping = {
    0.0: 'WolkenKlar',
    1.0: 'WolkenKlar',          
    2.0: 'WolkenKlar',
    3.0: 'WolkenKlar',
    4.0: 'WolkenBewoelkt',
    5.0: 'WolkenBewoelkt',
    6.0: 'WolkenBewoelkt',
    7.0: 'WolkenBewoelkt',
    8.0: 'WolkenBedeckt',               
}

df_merged['Bewoelkung'] = df_merged['Bewoelkung'].map(Bewoelkung_mapping)


# One-Hot-Encoding auf die neuen Kategorien
df_merged = pd.get_dummies(df_merged, columns=['Bewoelkung'], dtype=int, prefix='Bewoelkung', prefix_sep='')

print("Bewoelkung-Verteilung:")


print(df_merged.head())

Bewoelkung-Verteilung:
        id      Datum      Umsatz  Temperatur  Windgeschwindigkeit  \
0  1307011 2013-07-01  148.828353     17.8375                 15.0   
1  1307013 2013-07-01  201.198426     17.8375                 15.0   
2  1307014 2013-07-01   65.890169     17.8375                 15.0   
3  1307015 2013-07-01  317.475875     17.8375                 15.0   
4  1307012 2013-07-01  535.856285     17.8375                 15.0   

   KielerWoche Feiertage  WettercodeDunst  WettercodeExtremwetter  \
0          0.0         0                0                       0   
1          0.0         0                0                       0   
2          0.0         0                0                       0   
3          0.0         0                0                       0   
4          0.0         0                0                       0   

   WettercodeKlar  ...  Juni  Juli  August  September  Oktober  November  \
0               0  ...     0     1       0          0        0   

#### Feiertage

In [59]:
# Debug-Ausgabe
print("dtype vor Änderung:", df_merged['Feiertage'].dtype)

# Falls noch kein category dtype: konvertieren
if not pd.api.types.is_categorical_dtype(df_merged['Feiertage']):
    df_merged['Feiertage'] = df_merged['Feiertage'].astype('category')


print("dtype nach Änderung:", df_merged['Feiertage'].dtype)
print(df_merged['Feiertage'].value_counts())

dtype vor Änderung: object
dtype nach Änderung: category
Feiertage
0                            11043
Tag der Deutschen Einheit       30
Christi Himmelfahrt             25
Ostermontag                     25
Pfingstmontag                   20
Erster Mai                       5
Name: count, dtype: int64


/tmp/ipykernel_31496/1346139378.py:5: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df_merged['Feiertage']):


In [60]:
# Bewoelkung in Kategorien gruppieren mit Mapping

Feiertage_mapping = {
    0: 'Keiner',
    'Heiligabend': 'Heiligabend',                  
    'Tag der Deutschen Einheit': 'TagDerDeutschenEinheit',    
    'Silvester': 'Silvester',                   
    'Ostermontag': 'Ostermontag',                    
    'Christi Himmelfahrt': 'ChristiHimmelfahrt',            
    'Pfingstmontag': 'Pfingstmontag',                  
    'Erster Mai': 'ErsterMai',                              
}

df_merged['Feiertage'] = df_merged['Feiertage'].map(Feiertage_mapping)


# One-Hot-Encoding auf die neuen Kategorien
df_merged = pd.get_dummies(df_merged, columns=['Feiertage'], dtype=int, prefix='Feiertag', prefix_sep='')


print(df_merged.head())

        id      Datum      Umsatz  Temperatur  Windgeschwindigkeit  \
0  1307011 2013-07-01  148.828353     17.8375                 15.0   
1  1307013 2013-07-01  201.198426     17.8375                 15.0   
2  1307014 2013-07-01   65.890169     17.8375                 15.0   
3  1307015 2013-07-01  317.475875     17.8375                 15.0   
4  1307012 2013-07-01  535.856285     17.8375                 15.0   

   KielerWoche  WettercodeDunst  WettercodeExtremwetter  WettercodeKlar  \
0          0.0                0                       0               0   
1          0.0                0                       0               0   
2          0.0                0                       0               0   
3          0.0                0                       0               0   
4          0.0                0                       0               0   

   WettercodeNachSchlechtemWetter  ...  Dezember  BewoelkungWolkenBedeckt  \
0                               1  ...         0   

#### Fastenzeit importieren

In [61]:
df_fastenzeit = pd.read_csv('data/fastenzeit.csv')
df_fastenzeit['Datum'] = pd.to_datetime(df_fastenzeit['Datum'], errors='coerce')
df_merged = pd.merge(df_merged, df_fastenzeit[['Datum', 'Fastenzeit']], on='Datum', how='left')
print(df_merged.columns)

Index(['id', 'Datum', 'Umsatz', 'Temperatur', 'Windgeschwindigkeit',
       'KielerWoche', 'WettercodeDunst', 'WettercodeExtremwetter',
       'WettercodeKlar', 'WettercodeNachSchlechtemWetter', 'WettercodeNebel',
       'WettercodeRegen', 'WettercodeSchnee', 'WarengruppeBread',
       'WarengruppeRolls', 'WarengruppeCroissants', 'WarengruppeConfectionery',
       'WarengruppeCake', 'WarengruppeSeasonalBread', 'WochentagFreitag',
       'WochentagSamstag', 'WochentagSonntag', 'WochentagUnterDerWoche',
       'Januar', 'Februar', 'Maerz', 'April', 'Mai', 'Juni', 'Juli', 'August',
       'September', 'Oktober', 'November', 'Dezember',
       'BewoelkungWolkenBedeckt', 'BewoelkungWolkenBewoelkt',
       'BewoelkungWolkenKlar', 'FeiertagKeiner', 'FeiertagChristiHimmelfahrt',
       'FeiertagErsterMai', 'FeiertagOstermontag', 'FeiertagPfingstmontag',
       'FeiertagTagDerDeutschenEinheit', 'Fastenzeit'],
      dtype='object')


In [62]:
print(df_merged[['Datum', 'Fastenzeit']])
print(df_merged['Fastenzeit'].isna().sum())


           Datum  Fastenzeit
0     2013-07-01         0.0
1     2013-07-01         0.0
2     2013-07-01         0.0
3     2013-07-01         0.0
4     2013-07-01         0.0
...          ...         ...
49917 2019-07-30         NaN
49918 2019-07-30         NaN
49919 2019-07-30         NaN
49920 2019-07-30         NaN
49921 2019-07-30         NaN

[49922 rows x 2 columns]
1830


### Sicherung

In [63]:
df_merged.to_csv('data/df_final.csv', index=False)

## Data Splitting 

In [ ]:
# Dataset Splitting

df_merged = pd.read_csv('data/df_final_new.csv')


train = df_merged[(df_merged['Datum'] >= '2013-07-01') & (df_merged['Datum'] <= '2017-07-31')]
validation = df_merged[(df_merged['Datum'] >= '2017-08-01') & (df_merged['Datum'] <= '2018-07-31')]
test  = df_merged[(df_merged['Datum'] >= '2018-08-01') & (df_merged['Datum'] <= '2019-07-31')]

print("Train shape :", train.shape)
print("Validation shape :", validation.shape)
print("Test shape :", test.shape)

train.to_csv("data/train_data_new.csv", index=False)
validation.to_csv("data/validation_data_new.csv", index=False)
test.to_csv("data/test_data_new.csv", index=False)

Train shape : (38601, 45)
Validation shape : (9491, 45)
Test shape : (1830, 45)
